# Let's download wikipedia in english and basque and prepare for lm training

In [1]:
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm
basque = load_dataset("wikimedia/wikipedia", "20231101.eu", split="train")
basque = basque.shuffle(seed=42)

README.md: 0.00B [00:00, ?B/s]

20231101.eu/train-00000-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

20231101.eu/train-00001-of-00002.parquet:   0%|          | 0.00/114M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/416347 [00:00<?, ? examples/s]

In [2]:
basque

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 416347
})

In [3]:
english = load_dataset("wikimedia/wikipedia", "20231101.en", split="train", streaming=True)
english = english.shuffle(seed=42, buffer_size=1000000)
iterable_english = iter(english)
num_basque_items = len(basque)
english_list = []
for i in tqdm(range(num_basque_items)):
    next_english_entry = next(iterable_english)
    english_list.append(next_english_entry)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/416347 [00:00<?, ?it/s]

In [4]:
eng_dataset = Dataset.from_list(english_list)

In [5]:
eng_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 416347
})

In [6]:
num_total = num_basque_items
eng_dataset = eng_dataset.select(range(0,num_total))
bas_dataset = basque.select(range(0,num_total))
print(num_total)

416347


In [7]:
from transformers import AutoTokenizer
model_id = "HuggingFaceTB/SmolLM2-135M"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
eng_texts = [elt['text'] for elt in eng_dataset]
eng_tokenized = tokenizer(eng_texts, max_length=4096, padding=False, truncation=True, return_length=True)
eng_lengths = [len(elt) for elt in eng_tokenized['input_ids']]
sum(eng_lengths)

341816933

In [9]:
bas_texts = [elt['text'] for elt in bas_dataset]
bas_tokenized = tokenizer(bas_texts, max_length=4096, padding=False, truncation=True, return_length=True)
bas_lengths = [len(elt) for elt in bas_tokenized['input_ids']]
sum(bas_lengths)

198811459

In [10]:
eng_dataset_dict = eng_dataset.train_test_split(test_size=0.1, seed=42)
eng_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [ ]:
eng_dataset_test = eng_dataset_dict['test']

In [ ]:
bas_dataset_dict = bas_dataset.train_test_split(test_size=0.1, seed=42)
bas_dataset_test = bas_dataset_dict['test']
bas_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [13]:
from datasets import DatasetDict
upload_dataset = DatasetDict()
upload_dataset['train_en'] = eng_dataset_dict['train']
upload_dataset['test_en'] = eng_dataset_dict['test']
upload_dataset['train_ba'] = bas_dataset_dict['train']
upload_dataset['test_ba'] = bas_dataset_dict['test']
upload_dataset

DatasetDict({
    train_en: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test_en: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
    train_ba: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test_ba: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [14]:
upload_dataset.push_to_hub("Neelectric/wikipedia_20231101_en_eu_train_test_374k_41k")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/Neelectric/wikipedia_20231101_en_eu_train_test_374k_41k/commit/e8b8235e9f7683c304b08cc3ae1a2f147f2c1933', commit_message='Upload dataset', commit_description='', oid='e8b8235e9f7683c304b08cc3ae1a2f147f2c1933', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Neelectric/wikipedia_20231101_en_eu_train_test_374k_41k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Neelectric/wikipedia_20231101_en_eu_train_test_374k_41k'), pr_revision=None, pr_num=None)

In [ ]:
# mixed = eng_dataset_dict
# mixed['test'] = bas_dataset_dict['test']
# mixed

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [ ]:
# mixed.push_to_hub(("Neelectric/wikipedia_20231101.374k_en_41k_eu"))

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/Neelectric/wikipedia_20231101.374k_en_41k_eu/commit/4c3c4c89ef072a16329e6926b4484676c5659064', commit_message='Upload dataset', commit_description='', oid='4c3c4c89ef072a16329e6926b4484676c5659064', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Neelectric/wikipedia_20231101.374k_en_41k_eu', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Neelectric/wikipedia_20231101.374k_en_41k_eu'), pr_revision=None, pr_num=None)